In [1]:
import os
import datetime
import torch
import random
import pickle
import pandas as pd

#os.chdir(os.path.join(os.getcwd(), 'LAS Model'))
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data import SpeechDataset, AudioDataLoader
from listener import Listener
from attend_and_spell import AttendAndSpell
from seq2seq import Seq2Seq
from utils import  train

/home/computermaestro/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/computermaestro/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/computermaestro/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/computermaestro/.local/lib/python3.6/

In [2]:
def get_chars(lang, save_file=None, train_df=None):
    if lang=='eng':
        chars = ['<sos>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', \
                'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', \
                'y', 'z', ' ', "'", '<eos>', '<pad>']
    elif lang=='chinese':            
        try:
            with open(save_file, 'rb') as f:
                chars = pickle.load(f) # load file
        except FileNotFoundError:
            chars = [' ', '<sos>']
            for idx in range(train_df.shape[0]):
                _, sent = train_df.iloc[idx]
                for c in sent:
                    if c not in chars:
                        chars.append(c)
            chars = chars + ['<eos>', '<pad>']
            with open(save_file, 'wb') as f:
                pickle.dump(chars, f) # save file
    else:
        raise NotImplementedError
    
    print('Number of chars', len(chars))
    return chars


# Used when each sentence is in a separate text file
def make_train_df(dataset_dir):
    data = []
    files = os.listdir(dataset_dir)
    for f in files:
        if '.txt' in f:
            with open(os.path.join(dataset_dir, f), 'r') as text:
                data.append((f.replace('.txt', ''), text.readline()))
                
    train_df = pd.DataFrame(data, columns=['id', 'sent'])
    train_df.to_csv(os.path.join(dataset_dir, 'train_df.csv'), header=None)
    print(train_df.head())


# Used for ai_shell dataset, when all sentences are in a single text file
def read_transcrpt(transcript_dir):
    transcript_dir = '../../../Dataset'
    with open(os.path.join(transcript_dir, 'aishell_transcript_v0.8.txt')) as f:
        data_list = f.readlines()

    data = []
    for example in data_list:
        id_, sent = str(example.split(' ')[0]), str(' '.join(example.split(' ')[1:-1])) # -1 to remove '\n'
        data.append((id_, sent))

    print('Num examples:', len(data))
    data_df = pd.DataFrame(data, columns=['id', 'sent'])
    data_df.to_csv(os.path.join(transcript_dir, 'train_df.csv'))
    data_df.head()

In [3]:
dataset_dir = '../../../Dataset/data_aishell'
DEVICE = torch.device('cuda:1') if torch.cuda.is_available() else 'cpu'
print('DEVICE :', DEVICE)


train_df = pd.read_csv(os.path.join(dataset_dir, 'train_df.csv'), names=['id', 'sent'])
train_df = train_df.dropna(how='any')
print(train_df.head())
# test_df = pd.read_csv('test_df.csv', names=['id', 'sent'])

save_file = os.path.join('Trained Models', 'chars')
chars = get_chars('chinese', save_file, train_df)
char_to_token = {c:i for i,c in enumerate(chars)} 
token_to_char = {i:c for c,i in char_to_token.items()}
sos_token = char_to_token['<sos>']
eos_token = char_to_token['<eos>']
pad_token = char_to_token['<pad>']

tensorboard_dir = os.path.join('tb_summary')
train_dataset = SpeechDataset(train_df, dataset_dir, sos_token, char_to_token, eos_token)
train_loader = AudioDataLoader(pad_token, train_dataset, batch_size=32, shuffle=True, drop_last=True)

# #test_dataset = SpeechDataset(test_df, dataset_dir)
# #test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

DEVICE : cuda:1
                 id                        sent
0  BAC009S0002W0122     而 对 楼市 成交 抑制 作用 最 大 的 限
1  BAC009S0002W0123             也 成为 地方 政府 的 眼中
2  BAC009S0002W0124  自 六月 底 呼和浩特 市 率先 宣布 取消 限 购
3  BAC009S0002W0125                  各地 政府 便 纷纷
4  BAC009S0002W0126              仅 一 个 多 月 的 时间
Number of chars 4255


In [4]:
input_size = 128    # num rows in instagram
hidden_dim = 256    # 256*2 nodes in each LSTM
num_layers = 3
dropout = 0.1
layer_norm = False   
encoder = Listener(input_size, hidden_dim, num_layers, dropout=dropout, layer_norm=layer_norm)

hid_sz = 256
embed_dim = 100
vocab_size = len(chars)
decoder = AttendAndSpell(embed_dim, hid_sz, encoder.output_size, vocab_size)

hyperparams = {'input_size':input_size, 'hidden_dim':hidden_dim, 'num_layers':num_layers,
                'dropout':dropout, 'layer_norm':layer_norm, 'hid_sz':hid_sz, 'embed_dim':embed_dim}

criterion = nn.CrossEntropyLoss()
model = Seq2Seq(encoder, decoder, criterion, tf_ratio = 1.0, device=DEVICE).to(DEVICE)

In [ ]:
# optimizer = optim.ASGD(model.parameters(), lr=0.2)  # lr = 0.2 used in paper
optimizer = optim.Adadelta(model.parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.98)
log_interval = 5
print_interval = 40

epochs = 20
load = False
if load:
    saved_file = 'Trained Models/Training_2019-12-25 00:09:23.921978/las_model_6'
    model.load_state_dict(torch.load(saved_file))
    start_epoch = int(saved_file[-1]) + 1
    time = os.listdir(tensorboard_dir)[-1]  # use the last one
else:
    start_epoch = 0
    time = str(datetime.datetime.now())

save_dir = os.path.join('Trained Models', f'Training_{time}')
try:    
    os.mkdir(save_dir);
except FileExistsError:
    pass

summary_dir = os.path.join(tensorboard_dir, time)
writer = SummaryWriter(summary_dir)

# Saving hyperparmas
with open(os.path.join(save_dir, 'info.txt'), 'wb') as f:
    pickle.dump(hyperparams, f)

for epoch in range(start_epoch, epochs):
    print("\nTeacher forcing ratio:", model.tf_ratio)
    train(model, DEVICE, train_loader, optimizer, epoch, print_interval, writer, log_interval)
    scheduler.step()                                    # Decrease learning rate
    torch.save(model.state_dict(), os.path.join(save_dir, f'las_model_{epoch}'))
    model.tf_ratio = max(model.tf_ratio - 0.01, 0.8)    # Decrease teacher force ratio


Teacher forcing ratio: 1.0
Training, Logging: Mean loss of previous 40 batches 

Train Epoch: 0 [1248/141569 (1%)]	Mean Loss : 0.261120	 time 0:00:29.867798:
Train Epoch: 0 [2528/141569 (2%)]	Mean Loss : 0.261120	 time 0:00:26.448450:
Train Epoch: 0 [3808/141569 (3%)]	Mean Loss : 0.261119	 time 0:00:27.068087:
Train Epoch: 0 [5088/141569 (4%)]	Mean Loss : 0.261119	 time 0:00:26.559825:
Train Epoch: 0 [6368/141569 (4%)]	Mean Loss : 0.261118	 time 0:00:26.030298:
Train Epoch: 0 [7648/141569 (5%)]	Mean Loss : 0.261118	 time 0:00:26.209467:
Train Epoch: 0 [8928/141569 (6%)]	Mean Loss : 0.261117	 time 0:00:25.764259:
Train Epoch: 0 [10208/141569 (7%)]	Mean Loss : 0.261116	 time 0:00:26.393853:
Train Epoch: 0 [11488/141569 (8%)]	Mean Loss : 0.261115	 time 0:00:26.098833:
Train Epoch: 0 [12768/141569 (9%)]	Mean Loss : 0.261111	 time 0:00:27.027586:
Train Epoch: 0 [14048/141569 (10%)]	Mean Loss : 0.261104	 time 0:00:26.251614:
Train Epoch: 0 [15328/141569 (11%)]	Mean Loss : 0.261083	 time 0:0

Train Epoch: 0 [133088/141569 (94%)]	Mean Loss : 0.240397	 time 0:00:26.419357:
Train Epoch: 0 [134368/141569 (95%)]	Mean Loss : 0.239443	 time 0:00:27.348112:
Train Epoch: 0 [135648/141569 (96%)]	Mean Loss : 0.239922	 time 0:00:26.731726:
Train Epoch: 0 [136928/141569 (97%)]	Mean Loss : 0.239369	 time 0:00:27.618272:
Train Epoch: 0 [138208/141569 (98%)]	Mean Loss : 0.239488	 time 0:00:26.950408:
Train Epoch: 0 [139488/141569 (99%)]	Mean Loss : 0.239564	 time 0:00:27.492356:
Train Epoch: 0 [140768/141569 (99%)]	Mean Loss : 0.239501	 time 0:00:27.297704:

Teacher forcing ratio: 0.99
Training, Logging: Mean loss of previous 40 batches 

Train Epoch: 1 [1248/141569 (1%)]	Mean Loss : 0.239727	 time 0:00:26.455587:
Train Epoch: 1 [2528/141569 (2%)]	Mean Loss : 0.239919	 time 0:00:26.597855:
Train Epoch: 1 [3808/141569 (3%)]	Mean Loss : 0.239910	 time 0:00:26.136995:
Train Epoch: 1 [5088/141569 (4%)]	Mean Loss : 0.240045	 time 0:00:25.614074:
Train Epoch: 1 [6368/141569 (4%)]	Mean Loss : 0.2

Train Epoch: 1 [124128/141569 (88%)]	Mean Loss : 0.239491	 time 0:00:26.935020:
Train Epoch: 1 [125408/141569 (89%)]	Mean Loss : 0.239460	 time 0:00:26.961754:
Train Epoch: 1 [126688/141569 (89%)]	Mean Loss : 0.239741	 time 0:00:26.041079:
Train Epoch: 1 [127968/141569 (90%)]	Mean Loss : 0.239705	 time 0:00:26.484581:
Train Epoch: 1 [129248/141569 (91%)]	Mean Loss : 0.239702	 time 0:00:26.596082:
Train Epoch: 1 [130528/141569 (92%)]	Mean Loss : 0.239455	 time 0:00:27.664334:
Train Epoch: 1 [131808/141569 (93%)]	Mean Loss : 0.239427	 time 0:00:25.379719:
Train Epoch: 1 [133088/141569 (94%)]	Mean Loss : 0.239566	 time 0:00:25.088972:
Train Epoch: 1 [134368/141569 (95%)]	Mean Loss : 0.239501	 time 0:00:24.876092:
Train Epoch: 1 [135648/141569 (96%)]	Mean Loss : 0.239599	 time 0:00:24.434563:
Train Epoch: 1 [136928/141569 (97%)]	Mean Loss : 0.239832	 time 0:00:24.248281:
Train Epoch: 1 [138208/141569 (98%)]	Mean Loss : 0.239987	 time 0:00:24.194678:
Train Epoch: 1 [139488/141569 (99%)]	Mea

### TEST

In [ ]:
def decode_pred_sent(out):
    pred_sent = []
    for t in out:
        lol = t.max(dim=1)[1].item()
        pred_sent.append(token_to_char[lol])
    return ''.join(pred_sent)


def decode_true_sent(y):
    sent = []
    for t in y:
        sent.append(token_to_char[t.item()])
    return ''.join(sent)

In [ ]:
num_sent = 10
model.eval()
model.tf_ratio = 0.9

for _ in range(num_sent):
    
    idx = random.randint(0, train_df.shape[0])
    trial_dataset = SpeechDataset(train_df, dataset_dir, sos_token, char_to_token, eos_token)

    x, y = trial_dataset.__getitem__(idx)
    # plt.imshow(x[0,:,:].detach())

    # Model output
    target = y.unsqueeze(dim=0).to(DEVICE)
    data = x.permute(0, 2, 1).to(DEVICE)
    loss, output = model(data, target)
    print("True sent : ", decode_true_sent(y))
    print("Pred sent : ", decode_pred_sent(output))
    print("Loss :", loss.item())    
    print("\n")